<a href="https://colab.research.google.com/github/sunghyunjun/kaggle-otto-recsys/blob/main/otto_transformers4rec_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Check colab gcp, gpu environment

In [ ]:
# Check colab's gcp region
!curl ipinfo.io

{
  "ip": "108.59.82.129",
  "hostname": "129.82.59.108.bc.googleusercontent.com",
  "city": "Council Bluffs",
  "region": "Iowa",
  "country": "US",
  "loc": "41.2619,-95.8608",
  "org": "AS396982 Google LLC",
  "postal": "51502",
  "timezone": "America/Chicago",
  "readme": "https://ipinfo.io/missingauth"
}

In [ ]:
gcs_bucket = "sunghyun-kaggle-otto-us"
# gcs_bucket = "sunghyun-kaggle-otto-eu"
# gcs_bucket = "sunghyun-kaggle-otto-asia"

In [ ]:
!nvidia-smi -L

GPU 0: NVIDIA A100-SXM4-40GB (UUID: GPU-14042963-8783-888a-d82e-8257c78a1652)


In [ ]:
import multiprocessing
print(multiprocessing.cpu_count()) # Count the number of cores in a computer

12


# GCP personal setting for gcs, kaggle, neptune.ai

In [ ]:
# Authenticate to GCP
from google.colab import auth
auth.authenticate_user()

In [ ]:
# GCP secret manager, after run pip, notebook should be restarted
# !pip3 install --user google-cloud-secret-manager==2.5.0
!pip3 install --user google-cloud-secret-manager
from google.cloud import secretmanager


def get_secret(project_id, secret_name):
    """
    secret_name => To be replaced with your secret name
    project_id  => To be replaced with your GCP Project
    """

    # Create a Client:
    client = secretmanager.SecretManagerServiceClient()
    # secret_name = "kaggle" # => To be replaced with your secret name
    # project_id = 'otto-recsys' # => To be replaced with your GCP Project

    # Forge the path to the latest version of your secret with an F-string:
    resource_name = f"projects/{project_id}/secrets/{secret_name}/versions/latest" 

    # Get your secret :
    response = client.access_secret_version(request={"name": resource_name})
    secret_string = response.payload.data.decode('UTF-8')

    # Tada ! you secret is in the secret_string variable!
    return secret_string

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Install gcsfuse

In [ ]:
!pip install gcsfs --upgrade -q
!pip install psutil --upgrade -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 KB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.2/280.2 KB 6.4 MB/s eta 0:00:00


In [ ]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1210  100  1210    0     0  48400      0 --:--:-- --:--:-- --:--:-- 48400
OK
65 packages can be upgraded. Run 'apt list --upgradable' to see them.
The following NEW packages will be installed:
  gcsfuse
0 upgraded, 1 newly installed, 0 to remove and 65 not upgraded.
Need to get 13.3 MB of archives.
After this operation, 30.7 MB of additional disk space will be used.
Selecting previously unselected package gcsfuse.
(Reading database ... 129499 files and directories currently installed.)
Preparing to unpack .../gcsfuse_0.41.12_amd64.deb ...
Unpacking gcsfuse (0.41.12) ...
Setting up gcsfuse (0.41.12) ...


In [ ]:
!mkdir ./mount_checkpoint
!gcsfuse {gcs_bucket} ./mount_checkpoint

2023/01/31 03:42:58.962309 Start gcsfuse/0.41.12 (Go version go1.18.4) for app "" using mount point: /content/mount_checkpoint
2023/01/31 03:42:58.974092 Opening GCS connection...
2023/01/31 03:42:59.166603 Mounting file system "sunghyun-kaggle-otto-us"...
2023/01/31 03:42:59.166949 File system has been successfully mounted.


# Kaggle API credentials

In [ ]:
with open('kaggle.json', 'a') as fp:
    fp.write(get_secret("otto-recsys", "kaggle"))

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

kaggle.json


# Secret for Neptune logging

In [ ]:
import os
os.environ["NEPTUNE_PROJECT"] = "sunghyun.jun/otto-recsys"
os.environ["NEPTUNE_API_TOKEN"] = get_secret("otto-recsys", "neptune")
os.environ["NEPTUNE_ALLOW_SELF_SIGNED_CERTIFICATE"] = "TRUE"

# Download Kaggle dataset

In [ ]:
!kaggle kernels output sunghyunjun/otto-etl-nvtabular-cpu-candidates-orders -p dataset


Output file downloaded to dataset/candidates_orders.npy
Output file downloaded to dataset/test/_file_list.txt
Output file downloaded to dataset/test/_metadata
Output file downloaded to dataset/test/_metadata.json
Output file downloaded to dataset/test/part_0.parquet
Output file downloaded to dataset/test/part_1.parquet
Output file downloaded to dataset/test/part_2.parquet
Output file downloaded to dataset/test/part_3.parquet
Output file downloaded to dataset/test/part_4.parquet
Output file downloaded to dataset/test/part_5.parquet
Output file downloaded to dataset/test/part_6.parquet
Output file downloaded to dataset/test/part_7.parquet
Output file downloaded to dataset/test/part_8.parquet
Output file downloaded to dataset/test/schema.pbtxt
Output file downloaded to dataset/train/_file_list.txt
Output file downloaded to dataset/train/_metadata
Output file downloaded to dataset/train/_metadata.json
Output file downloaded to dataset/train/part_0.parquet
Output file downloaded to dataset/

# Install Transformers4Rec

In [ ]:
!pip install transformers4rec[pytorch,nvtabular]
!pip install -U nvtabular==1.3.3
!pip install -U pytorch_lightning==1.8.0.post1
!pip install torchmetrics==0.10.0
!pip install neptune-client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.3/774.3 KB 12.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 76.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.6/133.6 KB 18.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.2/274.2 KB 33.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 KB 8.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting re

# Import Transformers4Rec

In [ ]:
import glob
import os
import warnings

import torch

warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=UserWarning)

from merlin_standard_lib import Schema, Tag
from transformers4rec import torch as tr
from transformers4rec.torch import Trainer
from transformers4rec.torch.ranking_metric import RecallAt
from transformers4rec.config.trainer import T4RecTrainingArguments

# Variables Setting

In [ ]:
DEBUG = False
RESUME = False
OUTPUT_PATH = "orders"

In [ ]:
MAX_SEQUENCE_LENGTH = 500
D_MODEL = 512

EPOCH = 4

if DEBUG:
    MAX_STEPS = 2000
else:
    MAX_STEPS = -1

In [ ]:
if DEBUG:
    train_path = [
        "./dataset/train/part_0.parquet",
        "./dataset/train/part_1.parquet",
        "./dataset/train/part_2.parquet",
        "./dataset/train/part_3.parquet",
    ]
    valid_path = "./dataset/valid/part_0.parquet"
    test_path = "./dataset/test/part_0.parquet"
else:
    train_path = sorted(glob.glob("./dataset/train/part_*.parquet"))
    valid_path = sorted(glob.glob("./dataset/valid/part_*.parquet"))
    test_path = sorted(glob.glob("./dataset/test/part_*.parquet"))
    
output_path = os.path.join("mount_checkpoint", OUTPUT_PATH)

# Create schema proto manually

In [ ]:
%%writefile schema.pb
feature {
  name: "aid-list"
  value_count {
    min: 1
    max: 500
  }
  type: INT
  int_domain {
    name: "aid-list"
    min: 1
    max: 657941
    is_categorical: true
  }
  annotation {
    tag: "item"
    tag: "categorical"
    tag: "list"
    tag: "item_id"
  }
}

Overwriting schema.pb


In [ ]:
schema = Schema().from_proto_text("schema.pb")

# Build model blocks

In [ ]:
inputs = tr.TabularSequenceFeatures.from_schema(
    schema,
    embedding_dims={"aid-list": D_MODEL},
    max_sequence_length=MAX_SEQUENCE_LENGTH,
    d_output=D_MODEL,
    masking="mlm",
)

In [ ]:
inputs

TabularSequenceFeatures(
  (to_merge): ModuleDict(
    (categorical_module): SequenceEmbeddingFeatures(
      (filter_features): FilterFeatures()
      (embedding_tables): ModuleDict(
        (aid-list): Embedding(657942, 512, padding_idx=0)
      )
    )
  )
  (_aggregation): ConcatFeatures()
  (projection_module): SequentialBlock(
    (0): DenseBlock(
      (0): Linear(in_features=512, out_features=512, bias=True)
      (1): ReLU(inplace=True)
    )
  )
  (_masking): MaskedLanguageModeling()
)

In [ ]:
# Define XLNetConfig class and set default parameters for HF XLNet config  
transformer_config = tr.XLNetConfig.build(
    d_model=D_MODEL, n_head=4, n_layer=2, total_seq_length=MAX_SEQUENCE_LENGTH
)

# Define the model block including: inputs, masking, projection and transformer block.
body = tr.SequentialBlock(
    inputs,
    tr.MLPBlock([D_MODEL]),
    tr.TransformerBlock(transformer_config, masking=inputs.masking)
)

In [ ]:
# Defines the evaluation top-N metrics and the cut-offs
metrics = [
    RecallAt(top_ks=[10, 20], labels_onehot=True)
]

# Define a head related to next item prediction task 
# Important: transformerc4rec v0.1.15
# NextItemPredictionTask should have parameter [hf_format=True]
head = tr.Head(
    body,
    tr.NextItemPredictionTask(weight_tying=True, hf_format=True, metrics=metrics),
    inputs=inputs,
)

# Get the end-to-end Model class 
model = tr.Model(head)

# Install extra module to resolve [ImportError Trainer]

In [ ]:
# ImportError Trainer
# cannot import name 'PROTOCOL_TLS' from 'urllib3.util.ssl_' (/usr/local/lib/python3.8/dist-packages/urllib3/util/ssl_.py)
# https://github.com/googlecolab/colabtools/issues/2499
!pip install --upgrade sagemaker

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Configure T4RecTrainingArguments and Trainer

In [ ]:
# Set hyperparameters for training 
train_args = T4RecTrainingArguments(
    data_loader_engine='pyarrow', 
    dataloader_drop_last=True,
    gradient_accumulation_steps=1,
    per_device_train_batch_size=192, # 128
    per_device_eval_batch_size=192, # 128
    evaluation_strategy="epoch",
    output_dir=output_path, 
    learning_rate=0.00075, # 0.0005
    # lr_scheduler_type='cosine', 
    # learning_rate_num_cosine_cycles_by_epoch=0.5 / EPOCH,
    num_train_epochs=EPOCH,
    max_sequence_length=MAX_SEQUENCE_LENGTH, 
    report_to="neptune",
    logging_steps=1000, # 500
    save_steps=10000,
    save_total_limit=10,
    no_cuda=False,
    fp16=True,
    dataloader_num_workers=24, # 4
    max_steps=MAX_STEPS,
)

PyTorch: setting up devices


In [ ]:
trainer = Trainer(
    model=model,
    args=train_args,
    schema=schema,
    compute_metrics=True,
)

Using amp half precision backend


In [ ]:
trainer.train_dataset_or_path = train_path
trainer.eval_dataset_or_path = valid_path

# Run Train

In [ ]:
trainer.train(resume_from_checkpoint = RESUME)

***** Running training *****
  Num examples = 10584384
  Num Epochs = 4
  Instantaneous batch size per device = 192
  Total train batch size (w. parallel, distributed & accumulation) = 192
  Gradient Accumulation steps = 1
  Total optimization steps = 220508
/usr/local/lib/python3.8/dist-packages/transformers/integrations.py:882: NeptuneDeprecationWarning: `init` is deprecated, use `init_run` instead. We'll end support of it in `neptune-client==1.0.0`. For details, see https://docs.neptune.ai/setup/neptune-client_1-0_release_changes
  self._neptune_run = self._neptune.init(
/usr/local/lib/python3.8/dist-packages/transformers/integrations.py:882: NeptuneDeprecationWarning: Parameter `run` is deprecated, use `with_id` instead. We'll end support of it in `neptune-client==1.0.0`.
  self._neptune_run = self._neptune.init(


https://app.neptune.ai/sunghyun.jun/otto-recsys/e/OT-79
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api/run#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


/usr/local/lib/python3.8/dist-packages/transformers/integrations.py:893: NeptuneDeprecationWarning: The object you're logging will be implicitly cast to a string. We'll end support of this behavior in `neptune-client==1.0.0`. To log the object as a string, use `str(object)` or `repr(object)` instead. For details, see https://docs.neptune.ai/setup/neptune-client_1-0_release_changes
  self._neptune_run["parameters"] = combined_dict


Epoch,Training Loss,Validation Loss,/next-item/recall At 10,/next-item/recall At 20,/loss
1,7.059400,No log,0.338892,0.372432,8.049314


Saving model checkpoint to mount_checkpoint/orders/checkpoint-10000
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to mount_checkpoint/orders/checkpoint-20000
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to mount_checkpoint/orders/checkpoint-30000
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to mount_checkpoint/orders/checkpoint-40000
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to mount_checkpoint/orders/checkpoint-50000
Trainer.model is not a `PreTrainedModel`, only saving its state dict.


KeyboardInterrupt: ignored

In [ ]:
# Killing Neptune asynchronous thread. All data is safe on disk and can be later synced manually using `neptune sync` command.
# !neptune sync

In [ ]:
trainer._save_model_and_checkpoint(save_model_class=True)

Saving model checkpoint to mount_checkpoint/orders/checkpoint-55246
Trainer.model is not a `PreTrainedModel`, only saving its state dict.


# Evaluate

In [ ]:
train_metrics = trainer.evaluate(eval_dataset=valid_path, metric_key_prefix='eval')


In [ ]:
for key in sorted(train_metrics.keys()):
    print(" %s = %s" % (key, str(train_metrics[key])))

 eval_/loss = 3.4871678352355957
 eval_/next-item/recall_at_10 = 0.700332760810852
 eval_/next-item/recall_at_20 = 0.7220437526702881
 eval_runtime = 1041.5372
 eval_samples_per_second = 1711.685
 eval_steps_per_second = 1.672
